In [1]:
import os.path
import numpy as np
from itertools import cycle
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Qt5Agg')
%matplotlib qt5

from pyofss import Domain, System, Gaussian, Fibre, Filter, Splitter
from pyofss import temporal_power, spectral_power, lambda_to_nu, nu_to_lambda
from pyofss import single_plot, map_plot, waterfall_plot, labels
from pyofss.field import energy

OpenclFibre is not activated, check if pyopencl and reikna modules are installed


In [2]:
from pyofss.domain import lambda_to_omega
from pyofss.modules.nonlinearity import calculate_gamma
gamma_passive = calculate_gamma(2.7*(10**(-20)), 37.8, lambda_to_omega(1035))
gamma_active = calculate_gamma(2.7*(10**(-20)), 29, lambda_to_omega(1035))
print("gamma_passive = ", gamma_passive)
print("gamma_active = ", gamma_active)

gamma_passive =  4.336221744085291
gamma_active =  5.652040756083586


In [3]:
domain = Domain(samples_per_bit = 2**14, bit_width = 200.0, total_bits = 1, centre_nu = lambda_to_nu(1028.0))
gaussian = Gaussian(name="initial_pulse", peak_power = 1, width = 10., C = 0, using_fwhm=True)
A = gaussian.generate(domain.t)
single_plot(domain.t, temporal_power(A), label="initial field")
E = energy(A, domain.t)
print("initial energy = ", E)


Generating single_plot...
initial energy =  0.010644670194312263


In [4]:
single_plot(domain.nu, spectral_power(A), label="initial field")


Generating single_plot...


In [5]:
dir = 'C:/Users/Karin/PycharmProjects/pyofss-master/amplification_fiber1'
sys = System( domain, A)
print("before fibre", np.max(spectral_power(sys.field)))
# sys.add(Filter(name="filter_1", width_nu=(Domain.vacuum_light_speed/(1040**2))*4, offset_nu=(lambda_to_nu(1040) - lambda_to_nu(1035)),
#                  m=1, channel=0, using_fwhm=False, type_filt = "reflected"))
# sys.add(Splitter(name="splitter", loss = 0.55))
# sys.add(Fibre(name="passive_fibre", length=0.8, gamma=gamma_passive, beta=np.array([22.2, 63.8]), total_steps=1000, method='ss_symmetric', useAmplification = False))
sys.add(Fibre(name="active_fibre_1", length=0.5*1e-3, gamma=gamma_active, total_steps=100, traces=100, beta=np.array([0, 0, 20]), method='ss_symmetric', small_signal_gain = 25, E_sat = 25.0, useAmplification = True))
# sys.add(Fibre(name="passive_fibre", length=0.8, gamma=gamma_passive, beta=np.array([22.2, 63.8]), total_steps=2000, method='ss_symmetric', local_error=0.005, useAmplification = False))
# sys.add(Splitter(name="splitter", loss = 0.6))
# sys.add(Filter(name="filter_2", width_nu=(Domain.vacuum_light_speed/(1030**2))*4, offset_nu=(lambda_to_nu(1030) - lambda_to_nu(1035)),
#                  m=1, channel=0, using_fwhm=False, type_filt = "reflected"))
# sys.add(Splitter(name="splitter", loss = 0.55))
# sys.add(Fibre(name="passive_fibre", length=0.8, gamma=gamma_passive, beta=np.array([22.2, 63.8]), self_steepening=False, raman_scattering=True,
#                  total_steps=100, method='ss_symmetric', local_error=0.005, useAmplification = False))
# sys.add(Fibre(name="active_fibre_2", length=2.5, gamma=gamma_active, beta=np.array([24.9, 59]), self_steepening=False, raman_scattering=True,
#                  total_steps=100, method='ss_symmetric', local_error=0.005, small_signal_gain = 25, E_sat = 36.6, useAmplification = True))
# sys.add(Fibre(name="passive_fibre", gamma=gamma_passive, beta=np.array([22.2, 63.8]), self_steepening=False, raman_scattering=True,
#                  total_steps=100, method='ss_symmetric', local_error=0.005, useAmplification = False))
# sys.add(Splitter(name="splitter", loss = 0.6))

# for i in range(10):      
sys.run()

before fibre 0.005665450177283992
Directory:  C:/Users/Karin/PycharmProjects/pyofss-master/amplification_fiber1  found...


c:\Users\Karin\PycharmProjects\pyofss-master\pyofss\modules\storage.py:97: UserWarning: Directory is not empty
  warnings.warn("Directory is not empty")
  2%|▏         | 2/100 [00:00<00:06, 15.05it/s]

Caching linear factor


100%|██████████| 100/100 [00:07<00:00, 13.75it/s]


In [26]:
E = energy(sys.field, domain.t)
print("total pulse energy after fiber, Etot = ", E)

total pulse energy after fiber, Etot =  2.9882362741928596


In [27]:
E = energy(A, domain.t)
print("total pulse energy before fiber, Etot = ", E)

total pulse energy before fiber, Etot =  0.010644670194312263


In [6]:
dir = 'C:/Users/Karin/PycharmProjects/pyofss-master/amplification_fiber1'
storage = sys['active_fibre_1'].stepper.storage
(x, y, z) = storage.get_plot_data()
storage.draw_animation()

Directory:  C:/Users/Karin/PycharmProjects/pyofss-master/amplification_fiber1  found...
Directory is empty OK


In [ ]:
storage.save_all_storage_to_dir(dir = dir)


storage.draw_animation()

In [7]:
single_plot(domain.t, temporal_power(sys.field), label="after active fiber")
print("peak pulse energy after fibre, Emax = ", np.max(temporal_power(sys.field)))


Generating single_plot...
peak pulse energy after fibre, Emax =  280.7269000718777


In [8]:
single_plot(domain.nu, spectral_power(sys.field), label="after active fiber")
print("peak pulse energy (spectral) after fibre, Emax(spec) = ", np.max(spectral_power(sys.field)))


Generating single_plot...
peak pulse energy (spectral) after fibre, Emax(spec) =  1.5866124967318762
